In [1]:
import sys

print(sys.version)

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


In [ ]:
!pip install konlpy

In [5]:
import pandas as pd
from konlpy.tag import Kkma, Okt
from konlpy.tag import Okt
from tqdm import tqdm

### 형태소 분리

In [65]:
# 형태소 분리
def splitByMorphs(text):
  okt = Okt()
  morph_list = okt.morphs(text, stem=True) # 형태소로 분리, 원형
  print("형태소 분리 결과:", morph_list)
  searchMatchingKeywords(morph_list)

### 이모지 데이터셋 전처리

In [47]:
# 데이터 불러오기
data = pd.read_csv('./text-replacement-tags - Emoji.csv', 
                   sep=',', 
                   names=['Emoji', 'Category', 'Emotion', 'Emotion Level', 'tags'], 
                   keep_default_na=False)
data

,Emoji,Category,Emotion,Emotion Level,tags
0,emoji,category,감정,감정 세기(0-2),"tags -> .split(',')"
1,( • - •),emoji,neutral,0,"무표정,한심,o_o"
2,(๑>ᴗ<๑),emoji,happy,2,"애교,><,뀨,뿌잉"
3,( ღ'ᴗ'ღ ),emoji,happy,1,"하트,헤헤"
4,◟( ˘ ³˘)◞ ♡,emoji,happy,1,하트
...,...,...,...,...,...
389,૮(^*ﻌ*^)ა,emoji,happy,,강아지
390,੧˵ ಠ ᴥ ಠ ˵ノ,emoji,,,강아지
391,(っ⊂•ᴥ•⊃)っ,emoji,neutral,,강아지
392,∈･^ミ┬┬~,emoji,neutral,,강아지


In [48]:
# 필요없는 column 제거
data = data.drop(columns=['Category', 'Emotion', 'Emotion Level'], axis=0)
data

,Emoji,tags
0,emoji,"tags -> .split(',')"
1,( • - •),"무표정,한심,o_o"
2,(๑>ᴗ<๑),"애교,><,뀨,뿌잉"
3,( ღ'ᴗ'ღ ),"하트,헤헤"
4,◟( ˘ ³˘)◞ ♡,하트
...,...,...
389,૮(^*ﻌ*^)ა,강아지
390,੧˵ ಠ ᴥ ಠ ˵ノ,강아지
391,(っ⊂•ᴥ•⊃)っ,강아지
392,∈･^ミ┬┬~,강아지


In [49]:
# 첫 줄 제거 후 인덱스 초기화
data = data.drop([0])
data = data.reset_index()
data = data.drop(columns=['index'], axis=0)
data

,Emoji,tags
0,( • - •),"무표정,한심,o_o"
1,(๑>ᴗ<๑),"애교,><,뀨,뿌잉"
2,( ღ'ᴗ'ღ ),"하트,헤헤"
3,◟( ˘ ³˘)◞ ♡,하트
4,(•ө•)♡,하트
...,...,...
388,૮(^*ﻌ*^)ა,강아지
389,੧˵ ಠ ᴥ ಠ ˵ノ,강아지
390,(っ⊂•ᴥ•⊃)っ,강아지
391,∈･^ミ┬┬~,강아지


In [54]:
# tag가 여러 개 있을 경우 column 분리

for i in range(len(data)):
  temp = data.iloc[i, 1]

  if temp:
    tag_list = temp.split(',')

    for j in range(len(tag_list)):
      s = "data.loc[%d, 'tag_%d'] = tag_list[%d]"%(i, j+1, j)
      exec(s)

data = data.drop(columns=['tags'], axis=0)
data

,Emoji,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,tag_10
0,( • - •),무표정,한심,o_o,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(๑>ᴗ<๑),애교,><,뀨,뿌잉,NaN,NaN,NaN,NaN,NaN,NaN
2,( ღ'ᴗ'ღ ),하트,헤헤,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,◟( ˘ ³˘)◞ ♡,하트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(•ө•)♡,하트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
388,૮(^*ﻌ*^)ა,강아지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,੧˵ ಠ ᴥ ಠ ˵ノ,강아지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390,(っ⊂•ᴥ•⊃)っ,강아지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
391,∈･^ミ┬┬~,강아지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 이모지 데이터셋에서 동일 키워드 검색

In [99]:
# 이모지 데이터셋에서 동일 키워드 검색
def searchMatchingKeywords(morphs):
  for word in morphs:
    search_result = data.index[(data['tag_1'] == word) |
                               (data['tag_2'] == word) | 
                               (data['tag_3'] == word) |
                               (data['tag_4'] == word) |
                               (data['tag_5'] == word) | 
                               (data['tag_6'] == word) | 
                               (data['tag_7'] == word) |
                               (data['tag_8'] == word) | 
                               (data['tag_9'] == word) | 
                               (data['tag_10'] == word)
                                ].tolist()
    if search_result:
      print("'{}' 키워드로 추천하는 이모지:".format(word))
      for index in search_result:
        print(data.loc[index,'Emoji'])
      print()

### 모델 활용

In [100]:
splitByMorphs("나한테 애교 부리는 하트 보내줘 한심해 뿌잉 헤헤 나 진짜 속상해")

형태소 분리 결과: ['나', '한테', '애교', '부리다', '하트', '보내다', '한심하다', '뿌잉', '헤헤', '나', '진짜', '속상하다']
'애교' 키워드로 추천하는 이모지:
(๑>ᴗ<๑)
҉ ٩(๑>ω<๑)۶҉
o(*'▽'*)/☆ﾟ’
٩(๑>∀<๑)۶
( ๑˃̶ ꇴ ˂̶)♪⁺
(๑˃̵ᴗ˂̵)و

'하트' 키워드로 추천하는 이모지:
( ღ'ᴗ'ღ )
◟( ˘ ³˘)◞ ♡
(•ө•)♡
(ฅ•ω•ฅ)♡
( ்́ꇴ ்̀)♡
ღ'ᴗ'ღ
๑❤‿❤๑
(๑˃̵ᴗ˂̵)و ♡
꒰◍ˊ◡ˋ꒱੭ु⁾⁾♡
(*ฅ́˘ฅ̀*)♡
( ˇ͈ᵕˇ͈ ) ¨̮♡⃛
(*ˊૢᵕˋૢ*)ෆ
( ˃̆ૢ௰˂̆ૢഃ ) ლ
٩(๑• ₃ -๑)۶♥
ʕ•ﻌ•ʔ ♡
ღ˘‿˘ற꒱
ෆ╹ .̮ ╹ෆ
ლ|'ー'ლ|
(✿◖◡​◗)❤

'뿌잉' 키워드로 추천하는 이모지:
(๑>ᴗ<๑)
҉ ٩(๑>ω<๑)۶҉
o(*'▽'*)/☆ﾟ’
٩(๑>∀<๑)۶
( ๑˃̶ ꇴ ˂̶)♪⁺
(๑˃̵ᴗ˂̵)و

'헤헤' 키워드로 추천하는 이모지:
( ღ'ᴗ'ღ )
ღ'ᴗ'ღ
ლ( ╹ ◡ ╹ ლ)
(❁´▽`❁)
(❀╹◡╹)
(๑･̑◡･̑๑)
◡'✿
✦‿✦
๑･̑◡･̑๑
๑❛◡ુ❛๑
꒰( ˵¯͒ꇴ¯͒˵ )꒱
◕‿◕✿
๑•‿•๑
๑◕‿‿◕๑
✿˘◡˘✿
(▰˘◡˘▰)
˘◡˘
(๑′ᴗ‵๑)
(∗❛⌄❛∗)
(๑´◡ુ`๑)
p(´∇｀)q
(*´ ワ `*)“
ξ(✿ ❛‿❛)ξ
(๑・‿・๑)
✧*.◟(ˊᗨˋ)◞.*✧
✺◟(∗❛ัᴗ❛ั∗)◞✺
ᕕ( ᐛ )ᕗ
(╹౪╹*๑)
(๑╹∀╹๑)
(*ૂ❛ัᴗ❛ั*ૂ)
٩( 'ω' )و
へ(￣∇￣へ)

